## Retrieve MTBS by state from Source coop to test Dask and Docker
For more information: https://source.coop/repositories/cboettig/fire/description

In [7]:
# Import necessary functions 
from utils.source_coop_utils import initialize_dask_cluster,get_usgs_data,get_mtbs_shp,create_wildfire_severity_map
import matplotlib.pyplot as plt
import os
import boto3
from dotenv import load_dotenv

### Initialize a Dask cluster for distributed computing

This helps in handling large datasets and parallel computation efficiently.

In [ ]:
# Example usage
client = initialize_dask_cluster()

### Retrieve the AWS Access Key ID and AWS Secret Access Key from the environment variables.

AWS_ACCESS_KEY_ID = "Your AWS_ACCESS_KEY_ID Here "

AWS_SECRET_ACCESS_KEY = "Your AWS_ACCESS_KEY_ID Here")

In [2]:
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")#  replace AWS_ACCESS_KEY_ID with your access key 
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")#  replace AWS_SECRET_ACCESS_KEY with your access key 

In [ ]:
# Checking AWS Key 
AWS_ACCESS_KEY_ID

### Create an S3 client using the boto3 library with custom authentication and endpoint settings.

In [3]:
s3_client = boto3.client(
    's3',                             # Specify the AWS service: 's3' (Simple Storage Service).
    aws_access_key_id=AWS_ACCESS_KEY_ID,       # Provide the AWS Access Key ID for authentication.
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY, # Provide the AWS Secret Access Key for authentication.
    endpoint_url='https://data.source.coop'    # Specify a custom S3-compatible endpoint URL (non-AWS S3 service).
)

### You can skip making new directory if you directory already exists

In [12]:
# creating a new directory
os.makedirs('./datas', exist_ok=True)


### Define a local path where data will be stored or accessed.

In [4]:
local_path = "./data"

In [ ]:
# Define the name of the file to be fetched,
file_name= "usgs-mtbs"
# Fetch USGS data using the `get_usgs_data` function.
usgs_ddf = get_usgs_data(file_name, s3_client,local_path)

In [ ]:
# Define the name of the file to be fetched,
file_name="mtbs_perims_DD"
# Fetch MTBS (Monitoring Trends in Burn Severity) shapefile data using the `get_mtbs_shp` function.
mtbs_shp_ddf=get_mtbs_shp(file_name, s3_client,local_path)

In [ ]:
mtbs_shp_ddf

### Create a wildfire severity map using the MTBS shapefile data.

In [ ]:
map_object = create_wildfire_severity_map(mtbs_shp_ddf)
map_object

### This section filters wildfire incidents, groups them by state, counts the occurrences, and visualizes the results as a bar chart showing the number of wildfires by state.

In [ ]:
# Filter the data for Wildfire incidents
wildfire_ddf = mtbs_shp_ddf[mtbs_shp_ddf['Incid_Type'] == 'Wildfire']

# Extract state abbreviation from the 'Event_ID' (first two characters)
wildfire_ddf['State'] = wildfire_ddf['Event_ID'].str[:2]

# Group by 'State' and count the number of wildfires
wildfires_by_state = wildfire_ddf.groupby('State').size().compute()

# Convert to a DataFrame for better display
wildfires_by_state_df = wildfires_by_state.reset_index()
wildfires_by_state_df.columns = ['State', 'Wildfire Count']

# Sort the DataFrame by Wildfire Count in descending order
wildfires_by_state_df = wildfires_by_state_df.sort_values(by='Wildfire Count', ascending=False)

# Plot the data as a bar chart
plt.figure(figsize=(15, 8))
plt.bar(wildfires_by_state_df['State'], wildfires_by_state_df['Wildfire Count'], color='orange')

# Add titles and labels
plt.title('Number of Wildfires by State', fontsize=16)
plt.xlabel('State', fontsize=14)
plt.ylabel('Wildfire Count', fontsize=14)

# Add value labels on top of the bars
for index, value in enumerate(wildfires_by_state_df['Wildfire Count']):
    plt.text(index, value + 5, str(value), ha='center', va='bottom', fontsize=10)

# Show the plot
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()